# **Notebook zur Evaluation - Trajektorien**
***

In [180]:
import csv
import numpy as np
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.cm as cm
import plotly.graph_objects as go

***
### **Trajektorien CSVs laden**

In [181]:
hook_num = 1
v = 10
hook_model = 'A'
frame = '_work'
# frame = ''
num_interp_points = 301

hole_width = 8
hole_height = 12

hook_width = 1.8
hook_height = 5.4

In [182]:
def load_csv_to_trajectory(filepath):
    """Lädt die CSV und stellt Trajektorie-Liste wieder her"""
    trajectory = []
    with open(filepath, mode='r', newline='') as file:
        reader = csv.reader(file)
        next(reader)  # Überspringe Kopfzeile

        for row in reader:
            numbers = list(map(float, row))
            trans = numbers[:3]
            rot = numbers[3:]
            trajectory.append((trans, rot))
    return trajectory

In [183]:
traj_optim_path = '/home/mo/Thesis/Evaluation/Trajektorientests/Modell' + hook_model + '/v_' + str(v) + '/' + str(hook_num) + '/trajectory_0' + str(frame) + '.csv'
traj_1_path = '/home/mo/Thesis/Evaluation/Trajektorientests/Modell' + hook_model + '/v_' + str(v) + '/' + str(hook_num) + '/trajectory_1' + str(frame) + '.csv'
traj_2_path = '/home/mo/Thesis/Evaluation/Trajektorientests/Modell' + hook_model + '/v_' + str(v) + '/' + str(hook_num) + '/trajectory_2' + str(frame) + '.csv'
traj_3_path = '/home/mo/Thesis/Evaluation/Trajektorientests/Modell' + hook_model + '/v_' + str(v) + '/' + str(hook_num) + '/trajectory_3' + str(frame) + '.csv'
traj_4_path = '/home/mo/Thesis/Evaluation/Trajektorientests/Modell' + hook_model + '/v_' + str(v) + '/' + str(hook_num) + '/trajectory_4' + str(frame) + '.csv'

In [184]:
trajectory_optim = load_csv_to_trajectory(traj_optim_path)
trajectory_1 = load_csv_to_trajectory(traj_1_path)
trajectory_2 = load_csv_to_trajectory(traj_2_path)
trajectory_3 = load_csv_to_trajectory(traj_3_path)
trajectory_4 = load_csv_to_trajectory(traj_4_path)

***
### **Interpolation der Trajektorie**

In [185]:
def interpolate_trajectory(traj, num_points):
    """
    Interpoliert eine Trajektorie auf `num_points` gleichmäßig verteilte Punkte.
    Nur die translatorischen Komponenten werden verwendet (3D Punkte).
    """
    positions = np.array([p[0] for p in traj])  # Nur Positionen
    rotations = np.array([p[1] for p in traj])  # Nur Rotationen

    # Abstände zwischen aufeinanderfolgenden Punkten
    deltas_trans = np.linalg.norm(np.diff(positions, axis=0), axis=1)
    cumulative_dist_trans = np.concatenate([[0], np.cumsum(deltas_trans)])
    target_distances_trans = np.linspace(0, cumulative_dist_trans[-1], num_points)

    interp_positions = []
    for dist in target_distances_trans:
        idx = np.searchsorted(cumulative_dist_trans, dist) - 1
        idx = np.clip(idx, 0, len(positions) - 2)
        t = (dist - cumulative_dist_trans[idx]) / (cumulative_dist_trans[idx + 1] - cumulative_dist_trans[idx])
        point = (1 - t) * positions[idx] + t * positions[idx + 1]
        interp_positions.append(point)

    # Rotationsteil
    if np.allclose(rotations, rotations[0]):  # wenn alle Rotationswerte gleich sind (Ansatz 2-4)
        interp_rotations = [rotations[0]] * num_points
    else:
        deltas_rot = np.linalg.norm(np.diff(rotations, axis=0), axis=1)
        cumulative_dist_rot = np.concatenate([[0], np.cumsum(deltas_rot)])
        target_distances_rot = np.linspace(0, cumulative_dist_rot[-1], num_points)

        interp_rotations = []
        for rot in target_distances_rot:
            idx = np.searchsorted(cumulative_dist_rot, rot) - 1
            idx = np.clip(idx, 0, len(rotations) - 2)
            denom = cumulative_dist_rot[idx + 1] - cumulative_dist_rot[idx]
            t = 0 if denom == 0 else (rot - cumulative_dist_rot[idx]) / denom
            point = (1 - t) * rotations[idx] + t * rotations[idx + 1]
            interp_rotations.append(point)
    return np.array(interp_positions), np.array(interp_rotations)


def fix_rpy(rpy):
    roll, pitch, yaw = rpy
    return [roll, -pitch, yaw]


def compute_rectangle_corners(trajectory, frame='', width=8, height=12):
    rectangles = []

    half_w = width / 2
    half_h = height / 2

    # Rechteck immer in der XZ-Ebene
    local_corners = np.array([
        [-half_w, 0, -half_h],
        [ half_w, 0, -half_h],
        [ half_w, 0,  half_h],
        [-half_w, 0,  half_h]
    ])

    for pos, rpy in trajectory:
        if frame == '_work':
            rpy = fix_rpy(rpy)

        rot = R.from_euler('xyz', rpy, degrees=True)
        rotated_corners = rot.apply(local_corners)
        translated_corners = rotated_corners + np.array(pos)
        rectangles.append(translated_corners.tolist())

    return rectangles

In [186]:
interp_positions_optim, interp_rotations_optim = interpolate_trajectory(trajectory_optim, num_interp_points)
trajectory_optim = list(zip(interp_positions_optim, interp_rotations_optim))
rectangles_optim = compute_rectangle_corners(trajectory_optim, frame=frame, width=hole_width, height=hole_height)
rectangles_optim_hooks = compute_rectangle_corners(trajectory_optim, frame=frame, width = hook_width, height = hook_height)

interp_positions_1, interp_rotations_1 = interpolate_trajectory(trajectory_1, num_interp_points)
trajectory_1 = list(zip(interp_positions_1, interp_rotations_1))
rectangles_1 = compute_rectangle_corners(trajectory_1, frame=frame, width=hole_width, height=hole_height)

interp_positions_2, interp_rotations_2 = interpolate_trajectory(trajectory_2, num_interp_points)
trajectory_2 = list(zip(interp_positions_2, interp_rotations_2))
rectangles_2 = compute_rectangle_corners(trajectory_2, frame=frame, width=hole_width, height=hole_height)

interp_positions_3, interp_rotations_3 = interpolate_trajectory(trajectory_3, num_interp_points)
trajectory_3 = list(zip(interp_positions_3, interp_rotations_3))
rectangles_3 = compute_rectangle_corners(trajectory_3, frame=frame, width=hole_width, height=hole_height)

interp_positions_4, interp_rotations_4 = interpolate_trajectory(trajectory_4, num_interp_points)
trajectory_4 = list(zip(interp_positions_4, interp_rotations_4))
rectangles_4 = compute_rectangle_corners(trajectory_4, frame=frame, width=hole_width, height=hole_height)

***
### **Berechnung der Abweichungen zwischen Trajektorie und Optimalwerten**

In [187]:
def absolute_traj_differences(traj1, traj2, frame='_work'):
    measurements = {}
    # Globale Diffs
    x_diffs, y_diffs, z_diffs = [], [], []
    roll_diffs, pitch_diffs, yaw_diffs = [], [], []
    euclidean_diffs = []

    # Lokale Diffs
    x_diffs_local, y_diffs_local, z_diffs_local = [], [], []

    if len(traj1) != len(traj2):
        print("ERROR - Length of trajectories unequal!")
    else:
        for idx in range(len(traj1)):
            pos1, rpy1 = traj1[idx]
            pos2, rpy2 = traj2[idx]

            pos1 = np.array(pos1)
            pos2 = np.array(pos2)
            rot1 = np.array(rpy1)
            rot2 = np.array(rpy2)

            pos_diff = pos2 - pos1
            rot_diff = rot2 - rot1

            # globale Differenzen
            euclidean_diffs.append(np.linalg.norm(pos_diff))
            x_diffs.append(pos_diff[0])
            y_diffs.append(pos_diff[1])
            z_diffs.append(pos_diff[2])
            roll_diffs.append(rot_diff[0])
            pitch_diffs.append(rot_diff[1])
            yaw_diffs.append(rot_diff[2])

            # lokale Transformierung
            if frame == '_work':
                r = R.from_euler('xyz', fix_rpy(rpy1), degrees=True)
            else:
                r = R.from_euler('xyz', rpy1, degrees=True)

            local_diff = r.inv().apply(pos_diff)
            x_diffs_local.append(local_diff[0])
            y_diffs_local.append(local_diff[1])
            z_diffs_local.append(local_diff[2])

    # globale Einträge
    measurements['x_diffs'] = x_diffs
    measurements['y_diffs'] = y_diffs
    measurements['z_diffs'] = z_diffs
    measurements['roll_diffs'] = roll_diffs
    measurements['pitch_diffs'] = pitch_diffs
    measurements['yaw_diffs'] = yaw_diffs
    measurements['euclidean_diffs'] = euclidean_diffs

    measurements['x_min'] = np.min(x_diffs)
    measurements['x_max'] = np.max(x_diffs)
    measurements['y_min'] = np.min(y_diffs)
    measurements['y_max'] = np.max(y_diffs)
    measurements['z_min'] = np.min(z_diffs)
    measurements['z_max'] = np.max(z_diffs)

    measurements['x_diffs_mean'] = np.mean(x_diffs)
    measurements['y_diffs_mean'] = np.mean(y_diffs)
    measurements['z_diffs_mean'] = np.mean(z_diffs)

    # lokale Einträge
    measurements['x_diffs_local'] = x_diffs_local
    measurements['y_diffs_local'] = y_diffs_local
    measurements['z_diffs_local'] = z_diffs_local

    measurements['x_min_local'] = np.min(x_diffs_local)
    measurements['x_max_local'] = np.max(x_diffs_local)
    measurements['y_min_local'] = np.min(y_diffs_local)
    measurements['y_max_local'] = np.max(y_diffs_local)
    measurements['z_min_local'] = np.min(z_diffs_local)
    measurements['z_max_local'] = np.max(z_diffs_local)

    measurements['x_diffs_mean_local'] = np.mean(x_diffs_local)
    measurements['y_diffs_mean_local'] = np.mean(y_diffs_local)
    measurements['z_diffs_mean_local'] = np.mean(z_diffs_local)

    return measurements

In [188]:
traj1_measurements = absolute_traj_differences(traj1 = trajectory_optim, traj2 = trajectory_1)
traj2_measurements = absolute_traj_differences(traj1 = trajectory_optim, traj2 = trajectory_2)
traj3_measurements = absolute_traj_differences(traj1 = trajectory_optim, traj2 = trajectory_3)
traj4_measurements = absolute_traj_differences(traj1 = trajectory_optim, traj2 = trajectory_4)

In [189]:
def plot_position_differences(measurements, hook_num, trajectory_process=0):
    # Globale Werte
    x = measurements['x_diffs']
    y = measurements['y_diffs']
    z = measurements['z_diffs']
    x_diffs_mean = measurements['x_diffs_mean']
    y_diffs_mean = measurements['y_diffs_mean']
    z_diffs_mean = measurements['z_diffs_mean']

    # Lokale Werte
    x_loc = measurements['x_diffs_local']
    y_loc = measurements['y_diffs_local']
    z_loc = measurements['z_diffs_local']
    x_loc_mean = measurements['x_diffs_mean_local']
    y_loc_mean = measurements['y_diffs_mean_local']
    z_loc_mean = measurements['z_diffs_mean_local']

    n = len(x)
    t = list(range(n))

    fig, axs = plt.subplots(2, 1, figsize=(14, 10))
    fig.suptitle(f"Positionsdifferenzen für Haken {hook_num} - Trajektorien-Ansatz {trajectory_process}", fontsize=16)

    ##### Subplot 1: Globale Differenzen
    axs[0].plot(t, x, label='x-Differenz', color='red')
    axs[0].axhline(x_diffs_mean, linestyle='--', color='red', label=rf"$\bar{{x}}$ = {x_diffs_mean:.3f}")
    axs[0].text(n * 0.2, x_diffs_mean + 0.05, rf"$\bar{{x}}$ = {x_diffs_mean:.3f}", color='red',
                fontsize=10, bbox=dict(facecolor='white', edgecolor='red', alpha=0.9))

    axs[0].plot(t, y, label='y-Differenz', color='green')
    axs[0].axhline(y_diffs_mean, linestyle='--', color='green', label=rf"$\bar{{y}}$ = {y_diffs_mean:.3f}")
    axs[0].text(n * 0.3, y_diffs_mean + 0.05, rf"$\bar{{y}}$ = {y_diffs_mean:.3f}", color='green',
                fontsize=10, bbox=dict(facecolor='white', edgecolor='green', alpha=0.9))

    axs[0].plot(t, z, label='z-Differenz', color='blue')
    axs[0].axhline(z_diffs_mean, linestyle='--', color='blue', label=rf"$\bar{{z}}$ = {z_diffs_mean:.3f}")
    axs[0].text(n * 0.4, z_diffs_mean + 0.05, rf"$\bar{{z}}$ = {z_diffs_mean:.3f}", color='blue',
                fontsize=10, bbox=dict(facecolor='white', edgecolor='blue', alpha=0.9))

    axs[0].set_title("Globale Positionsabweichung")
    axs[0].set_xlabel('Trajektorienpunkt')
    axs[0].set_ylabel('Differenz [mm]')
    axs[0].set_xlim(0, len(x))
    axs[0].legend()
    axs[0].grid(True)


    ##### Subplot 2: Lokale Differenzen
    axs[1].plot(t, x_loc, label='x-Differenz (lokal)', color='red')
    axs[1].axhline(x_loc_mean, linestyle=':', color='red', label=rf"$\bar{{x}}$ = {x_loc_mean:.3f}")
    axs[1].text(n * 0.2, x_loc_mean + 0.05, rf"$\bar{{x}}$ = {x_loc_mean:.3f}", color='red',
                fontsize=10, bbox=dict(facecolor='white', edgecolor='red', alpha=0.9))

    axs[1].plot(t, y_loc, label='y-Differenz (lokal)', color='green')
    axs[1].axhline(y_loc_mean, linestyle=':', color='green', label=rf"$\bar{{y}}$ = {y_loc_mean:.3f}")
    axs[1].text(n * 0.3, y_loc_mean + 0.05, rf"$\bar{{y}}$ = {y_loc_mean:.3f}", color='green',
                fontsize=10, bbox=dict(facecolor='white', edgecolor='green', alpha=0.9))

    axs[1].plot(t, z_loc, label='z-Differenz (lokal)', color='blue')
    axs[1].axhline(z_loc_mean, linestyle=':', color='blue', label=rf"$\bar{{z}}$ = {z_loc_mean:.3f}")
    axs[1].text(n * 0.4, z_loc_mean + 0.05, rf"$\bar{{z}}$ = {z_loc_mean:.3f}", color='blue',
                fontsize=10, bbox=dict(facecolor='white', edgecolor='blue', alpha=0.9))
    
    delta_width = (hole_width - hook_width) / 2
    delta_height = (hole_height - hole_width) / 2

    axs[1].plot(t, [delta_width] * len(x), linestyle='--', color='red', label='Kollisionsgrenze entlang X')
    axs[1].plot(t, [-delta_width] * len(x), linestyle='--', color='red')
    axs[1].plot(t, [delta_height] * len(x), linestyle='--', color='blue', label='Kollisionsgrenze entlang Z')
    axs[1].plot(t, [-delta_height] * len(x), linestyle='--', color='blue')

    axs[1].set_title("Lokale Positionsabweichung")
    axs[1].set_xlabel('Trajektorienpunkt')
    axs[1].set_ylabel('Differenz [mm]')
    axs[1].set_xlim(0, len(x))
    axs[1].legend()
    axs[1].grid(True)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [190]:
def show_measurements(dict, hook_num, trajectory_process=0):
    print("-------------------------------------------------------------")
    print(f"Messwerte für Trajektorie - Haken {hook_num} - Trajektorien-Ansatz {trajectory_process}\n")
    print("Absolute Distanzen\n")
    print(f"x max [WORK]: {dict['x_max']:.4f}mm - x max [LOCAL]: {dict['x_max_local']:.4f}mm")
    print(f"x min [WORK]: {dict['x_min']:.4f}mm - x min [LOCAL]: {dict['x_min_local']:.4f}mm")
    print(f"y max [WORK]: {dict['y_max']:.4f}mm - y max [LOCAL]: {dict['y_max_local']:.4f}mm")
    print(f"y min [WORK]: {dict['y_min']:.4f}mm - y min [LOCAL]: {dict['y_min_local']:.4f}mm")
    print(f"z max [WORK]: {dict['z_max']:.4f}mm - z max [LOCAL]: {dict['z_max_local']:.4f}mm")
    print(f"z min [WORK]: {dict['z_min']:.4f}mm - z min [LOCAL]: {dict['z_min_local']:.4f}mm\n")

    print(f"x mean [WORK]: {dict['x_diffs_mean']:.4f}mm - x mean [LOCAL]: {dict['x_diffs_mean_local']:.4f}mm")
    print(f"y mean [WORK]: {dict['y_diffs_mean']:.4f}mm - y mean [LOCAL]: {dict['y_diffs_mean_local']:.4f}mm")
    print(f"z mean [WORK]: {dict['z_diffs_mean']:.4f}mm - z mean [LOCAL]: {dict['z_diffs_mean_local']:.4f}mm\n")

    print()
    print(f"Maximale Euklidsche Distanz [WORK]: {np.max(dict['euclidean_diffs']):.4f}\n")
    
    print()
    print("Roll-Pitch-Yaw Abweichungen\n")
    print(f"Maximale Abweichung Roll (x) [WORK]: {np.max(dict['roll_diffs']):.4f}")
    print(f"Maximale Abweichung Pitch (y) [WORK]: {np.max(dict['pitch_diffs']):.4f}°")
    print(f"Maximale Abweichung Yaw (z) [WORK]: {np.max(dict['yaw_diffs']):.4f}°")

In [191]:
# plot_position_differences(traj1_measurements, hook_num, trajectory_process = 1)
show_measurements(traj1_measurements, hook_num, trajectory_process = 1)

# plot_position_differences(traj2_measurements, hook_num, trajectory_process = 2)
show_measurements(traj2_measurements, hook_num, trajectory_process = 2)

# plot_position_differences(traj3_measurements, hook_num, trajectory_process = 3)
show_measurements(traj3_measurements, hook_num, trajectory_process = 3)

# plot_position_differences(traj4_measurements, hook_num, trajectory_process = 4)
show_measurements(traj4_measurements, hook_num, trajectory_process = 4)

-------------------------------------------------------------
Messwerte für Trajektorie - Haken 1 - Trajektorien-Ansatz 1

Absolute Distanzen

x max [WORK]: -0.0000mm - x max [LOCAL]: -0.0000mm
x min [WORK]: -3.4803mm - x min [LOCAL]: -2.7453mm
y max [WORK]: -0.0000mm - y max [LOCAL]: 2.4143mm
y min [WORK]: -2.5660mm - y min [LOCAL]: -0.0000mm
z max [WORK]: 0.9498mm - z max [LOCAL]: -0.0000mm
z min [WORK]: -1.7655mm - z min [LOCAL]: -2.7989mm

x mean [WORK]: -2.3487mm - x mean [LOCAL]: -1.7216mm
y mean [WORK]: -0.8537mm - y mean [LOCAL]: 1.3000mm
z mean [WORK]: -0.6860mm - z mean [LOCAL]: -1.4904mm


Maximale Euklidsche Distanz [WORK]: 3.9350


Roll-Pitch-Yaw Abweichungen

Maximale Abweichung Roll (x) [WORK]: 26.8032
Maximale Abweichung Pitch (y) [WORK]: 15.8492°
Maximale Abweichung Yaw (z) [WORK]: 15.0250°
-------------------------------------------------------------
Messwerte für Trajektorie - Haken 1 - Trajektorien-Ansatz 2

Absolute Distanzen

x max [WORK]: -0.0000mm - x max [LOCAL

***
### **Kollisionsberechnung**

**Kollisionsberechnung mit punktförmigem Haken (unendlich dünn)**

In [192]:
def check_fits_through_hole(measurements, hole_width, hole_height):
    x_local = np.array(measurements['x_diffs_local'])
    z_local = np.array(measurements['z_diffs_local'])

    max_x_dev = hole_width / 2
    max_z_dev = hole_height / 2

    violations = []
    for i in range(len(x_local)):
        if abs(x_local[i]) > max_x_dev or abs(z_local[i]) > max_z_dev:
            violations.append(i)

    fits = len(violations) == 0
    return fits, violations

In [193]:
fits, collisions = check_fits_through_hole(traj1_measurements, hole_width=hole_width, hole_height=hole_height)
print("✅ Keine Kollision erkannt.") if fits else print(f"❌ Kollision an {len(collisions)} Stellen (Indizes):", collisions)

fits, collisions = check_fits_through_hole(traj2_measurements, hole_width=hole_width, hole_height=hole_height)
print("✅ Keine Kollision erkannt.") if fits else print(f"❌ Kollision an {len(collisions)} Stellen (Indizes):", collisions)

fits, collisions = check_fits_through_hole(traj3_measurements, hole_width=hole_width, hole_height=hole_height)
print("✅ Keine Kollision erkannt.") if fits else print(f"❌ Kollision an {len(collisions)} Stellen (Indizes):", collisions)

fits, collisions = check_fits_through_hole(traj4_measurements, hole_width=hole_width, hole_height=hole_height)
print("✅ Keine Kollision erkannt.") if fits else print(f"❌ Kollision an {len(collisions)} Stellen (Indizes):", collisions)


✅ Keine Kollision erkannt.
✅ Keine Kollision erkannt.
✅ Keine Kollision erkannt.
❌ Kollision an 187 Stellen (Indizes): [114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 

**Kollisionsberechnung mit modelliertem Haken**

In [194]:
def transform_hook_rectangles_to_local(rectangles_optim_hooks, trajectory, frame='_work'):
    """
    Transformiert eine Liste globaler Haken-Rechtecke in den lokalen Frame der gemessenen Trajektorie.

    :param rectangles_optim_hooks: Liste von Rechtecken (jeweils 4 Punkte), im globalen Raum
    :param trajectory: Liste von (Position, RPY) Posen der berechneten/messbasierten Trajektorie
    :param frame: '_work' oder 'xyz' – beeinflusst RPY-Interpretation
    :return: Liste von Rechtecken im LOCAL-Frame der jeweiligen Pose
    """
    rectangles_local = []

    for i, rect_global in enumerate(rectangles_optim_hooks):
        rect = np.array(rect_global)
        pos_meas, rpy_meas = trajectory[i]

        # Rotation aus gemessener Pose
        if frame == '_work':
            rot = R.from_euler('xyz', fix_rpy(rpy_meas), degrees=True)
        else:
            rot = R.from_euler('xyz', rpy_meas, degrees=True)

        translation = np.array(pos_meas)

        # Rechteck: global → lokal = verschieben + inverse Rotation
        rect_local = rot.inv().apply(rect - translation)
        rectangles_local.append(rect_local)

    return rectangles_local

In [195]:
rectangles_optim_hooks_local_1 = transform_hook_rectangles_to_local(rectangles_optim_hooks, trajectory_1)
rectangles_optim_hooks_local_2 = transform_hook_rectangles_to_local(rectangles_optim_hooks, trajectory_2)
rectangles_optim_hooks_local_3 = transform_hook_rectangles_to_local(rectangles_optim_hooks, trajectory_3)
rectangles_optim_hooks_local_4 = transform_hook_rectangles_to_local(rectangles_optim_hooks, trajectory_4)

In [196]:
def check_hook_rectangles_in_hole_local(measurements, rectangles_hooks_local, hole_width, hole_height):
    """
    Prüft, ob alle Hakenrechtecke im LOCAL-Frame vollständig innerhalb der Lochfläche liegen.
    
    :param measurements: Dict mit lokalen Abweichungen (für Längenvergleich)
    :param rectangles_hooks_local: Liste von Rechtecken im LOCAL-Frame (4 Punkte je Rechteck)
    :param hole_width: Lochbreite (X-Richtung)
    :param hole_height: Lochhöhe (Z-Richtung)
    :return: (bool: passt durch, list: kollidierende Indizes)
    """
    max_x = hole_width / 2
    max_z = hole_height / 2

    collisions = []

    for i, rect in enumerate(rectangles_hooks_local):
        rect = np.array(rect)

        # Prüfe, ob alle 4 Punkte innerhalb [-w/2, +w/2] × [-h/2, +h/2] liegen (X/Z)
        within_x = np.abs(rect[:, 0]) < max_x
        within_z = np.abs(rect[:, 2]) < max_z
        if not np.all(within_x & within_z):
            collisions.append(i)

    fits = len(collisions) == 0
    return fits, collisions

In [197]:
fits_1, collisions_1 = check_hook_rectangles_in_hole_local(traj1_measurements, rectangles_optim_hooks_local_1, hole_width, hole_height)
print("✅ Keine Kollision erkannt.") if fits_1 else print(f"❌ Kollision an {len(collisions_1)} Stellen (Indizes):", collisions_1)

fits_2, collisions_2 = check_hook_rectangles_in_hole_local(traj2_measurements, rectangles_optim_hooks_local_2, hole_width, hole_height)
print("✅ Keine Kollision erkannt.") if fits_2 else print(f"❌ Kollision an {len(collisions_2)} Stellen (Indizes):", collisions_2)

fits_3, collisions_3 = check_hook_rectangles_in_hole_local(traj3_measurements, rectangles_optim_hooks_local_3, hole_width, hole_height)
print("✅ Keine Kollision erkannt.") if fits_3 else print(f"❌ Kollision an {len(collisions_3)} Stellen (Indizes):", collisions_3)

fits_4, collisions_4 = check_hook_rectangles_in_hole_local(traj4_measurements, rectangles_optim_hooks_local_4, hole_width, hole_height)
print("✅ Keine Kollision erkannt.") if fits_4 else print(f"❌ Kollision an {len(collisions_4)} Stellen (Indizes):", collisions_4)

❌ Kollision an 105 Stellen (Indizes): [147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300]
❌ Kollision an 89 Stellen (Indizes): [184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 289, 290, 291

***
### **3D-Plot der Trajektorie und Export als HTML**

In [198]:
def visualize_rectangles_html(rectangles, connect_points=None, 
                              secondary_rectangles=None, secondary_legend_title=None,
                              hook_rectangles = None,
                              hook_num=0, trajectory_process=1, html_filename="/home/mo/Thesis/Evaluation/Trajektorientests/CSV/plot_interaktiv.html",
                              collision_list = None, plot_collisions = False):
    """
    Visualisiert Rechtecke und Trajektorien im 3D-Raum als interaktive HTML-Datei.
    
    :param rectangles: Liste von Rechtecken (jeweils 4 Punkte [x, y, z])
    :param figsize: Größe der Plotfläche (Breite, Höhe)
    :param connect_points: Optionales Tupel mit zwei Punkten [x, y, z], die verbunden werden sollen
    :param secondary_rectangles: Optionaler zweiter Satz Rechtecke (werden grau dargestellt)
    :param secondary_legend_title: Optionaler Legenden-Eintrag für die grauen Rechtecke
    :param html_filename: Der Name der HTML-Datei, die erstellt wird
    """
    fig = go.Figure()

    # Berechnete Trajektorie (farbig)
    cmap = plt.cm.get_cmap('tab20')
    centers = []

    if rectangles:
        for i, rect in enumerate(rectangles):
            # Verwende rgba-String für die Farbe (anstatt des RGBA-Tupels)
            '''
            color = f"rgba({int(cmap(i / len(rectangles))[0] * 255)}, " \
                    f"{int(cmap(i / len(rectangles))[1] * 255)}, " \
                    f"{int(cmap(i / len(rectangles))[2] * 255)}, 1.0)"
            '''
            color = 'blue'
            x, y, z = zip(*rect)

            fig.add_trace(go.Mesh3d(x=x, y=y, z=z, color=color, opacity=0.05, 
                                   name=f'P{i+1}'))
        
            center = np.mean(np.array(rect), axis=0)
            centers.append(center)

            text_label = ''
            # Füge den Text an den Punkten hinzu (mit entsprechender Position)
            fig.add_trace(go.Scatter3d(x=[center[0]], y=[center[1]], z=[center[2]], 
                                       mode='markers+text', marker=dict(size=3, color='blue'),
                                       text=text_label, textposition="top center", textfont=dict(size=12), showlegend=False))
            
            # Hinzufügen der Umrandung der Rechtecke mit einer Linie
            if i == 0 or i == (len(rectangles) - 1 ):
                for j in range(4):
                    x0, y0, z0 = rect[j]
                    x1, y1, z1 = rect[(j + 1) % 4]  # Nächster Punkt im Rechteck
                    fig.add_trace(go.Scatter3d(x=[x0, x1], y=[y0, y1], z=[z0, z1], 
                                               mode='lines', 
                                               line=dict(color=color, width=4),  # Keine opacity hier
                                               showlegend=False))
        
        # Linie durch die berechneten Mittelpunkte
        centers = np.array(centers)
        fig.add_trace(go.Scatter3d(x=centers[:, 0], y=centers[:, 1], z=centers[:, 2],
                                   mode='lines', line=dict(color='black', width=2), name='Berechnete Hakenlinie'))

        # Beschriftung der ersten und letzten Punkte
        if len(centers) > 0:
            fig.add_trace(go.Scatter3d(x=[centers[0, 0]], y=[centers[0, 1]], z=[centers[0, 2]],
                                       mode='text', text=['Pre-Position'], textposition="top center", textfont=dict(size=10), showlegend=False))
        if len(centers) > 2:
            fig.add_trace(go.Scatter3d(x=[centers[-1, 0]], y=[centers[-1, 1]], z=[centers[-1, 2]],
                                       mode='text', text=['Loslassen'], textposition="top center", textfont=dict(size=10), showlegend=False))
        
        # Tunnelkanten: Ecken durchziehen mit denselben Farben wie bei den Rechtecken
        num_corners = 4  # Immer 4 Ecken pro Rechteck
        for corner_idx in range(num_corners):
            x_line, y_line, z_line = [], [], []
            colors = []
            for i, rect in enumerate(rectangles):
                # Farbcode für das Rechteck
                colors.append(f"rgba({int(cmap(i / len(rectangles))[0] * 255)}, " \
                        f"{int(cmap(i / len(rectangles))[1] * 255)}, " \
                        f"{int(cmap(i / len(rectangles))[2] * 255)}, 1.0)")
                            
                x, y, z = rect[corner_idx]
                x_line.append(x)
                y_line.append(y)
                z_line.append(z)
            
            # Die Tunnelkanten mit dem Farbcode der Rechtecke zeichnen
            fig.add_trace(go.Scatter3d(x=x_line, y=y_line, z=z_line,
                                       mode='lines',
                                       line=dict(color='blue', width=3),
                                       showlegend=False))

    # Optimal-Trajektorie (grün) – Zweite Rechtecke
    if secondary_rectangles:
        for i,rect in enumerate(secondary_rectangles):
            x, y, z = zip(*rect)
            fig.add_trace(go.Mesh3d(x=x, y=y, z=z, color='rgba(0,255,0,1.0)', opacity=0.05, 
                                   name=secondary_legend_title or 'Optimale Trajektorie'))
            
            # Hinzufügen der Umrandung der Rechtecke mit einer Linie
            if i == 0 or i == (len(secondary_rectangles) - 1):
                for j in range(4):
                    x0, y0, z0 = rect[j]
                    x1, y1, z1 = rect[(j + 1) % 4]  # Nächster Punkt im Rechteck
                    fig.add_trace(go.Scatter3d(x=[x0, x1], y=[y0, y1], z=[z0, z1], 
                                               mode='lines', 
                                               line=dict(color='rgba(0,255,0,1.0)', width=3),  # Keine opacity hier
                                               showlegend=False))
            
            # Hinzufügen eines roten Punktes in der Mitte des Rechtecks
            center = np.mean(np.array(rect), axis=0)
            fig.add_trace(go.Scatter3d(x=[center[0]], y=[center[1]], z=[center[2]], 
                                       mode='markers', marker=dict(size=2, color='green'), 
                                       showlegend=False))
        
        # Mittelpunkte der sekundären Rechtecke sammeln
        secondary_centers = [np.mean(np.array(rect), axis=0) for rect in secondary_rectangles]
        secondary_centers = np.array(secondary_centers)

        # Linie durch die Mittelpunkte zeichnen
        fig.add_trace(go.Scatter3d(
            x=secondary_centers[:, 0], y=secondary_centers[:, 1], z=secondary_centers[:, 2],
            mode='lines',
            line=dict(color='rgba(0,255,0,1.0)', width=2),
            name='Optimale Hakenlinie'
        ))

    # Tunnelkanten Optimal-Trajektorie
    num_corners = 4  # Immer 4 Ecken pro Rechteck
    for corner_idx in range(num_corners):
        x_line, y_line, z_line = [], [], []
        colors = []
        for i, rect in enumerate(secondary_rectangles):                       
            x, y, z = rect[corner_idx]
            x_line.append(x)
            y_line.append(y)
            z_line.append(z)
        
        # Die Tunnelkanten mit dem Farbcode der Rechtecke zeichnen
        fig.add_trace(go.Scatter3d(x=x_line, y=y_line, z=z_line,
                                   mode='lines',
                                   line=dict(color='rgba(0,255,0,1.0)', width=3),
                                   showlegend=False))

    # Modellierte Hakenlinie
    if hook_rectangles:
        for i, rect in enumerate(hook_rectangles):
            x, y, z = zip(*rect)
            show_legend = i == 0  # Nur beim ersten Eintrag die Legende zeigen
            fig.add_trace(go.Mesh3d(
                x=x, y=y, z=z,
                color='rgba(0,0,0,1.0)',
                opacity=1.0,
                name='Haken (modelliert)',
                showlegend=show_legend
            ))
            # Umrandungslinien (optional showlegend=False hier)
            if i == 0 or i == (len(hook_rectangles) - 1):
                for j in range(4):
                    x0, y0, z0 = rect[j]
                    x1, y1, z1 = rect[(j + 1) % 4]
                    fig.add_trace(go.Scatter3d(
                        x=[x0, x1], y=[y0, y1], z=[z0, z1],
                        mode='lines',
                        line=dict(color='rgba(0,0,0,1.0)', width=3),
                        showlegend=False  # keine doppelten Legenden für Linien
                    ))

    # Einzeichnen von Spitze und Senke inkl. graue Verbindungslinie
    if connect_points:
        p1, p2 = np.array(connect_points[0]), np.array(connect_points[1])
        fig.add_trace(go.Scatter3d(x=[p1[0], p2[0]], y=[p1[1], p2[1]], z=[p1[2], p2[2]], 
                                   mode='lines', line=dict(color='gray', dash='dash'), name='Direkte Gerade Spitze -> Senke'))
        fig.add_trace(go.Scatter3d(x=[p1[0]], y=[p1[1]], z=[p1[2]], mode='markers', 
                                   marker=dict(size=8, color='magenta'), name='Spitze (berechnet)'))
        fig.add_trace(go.Scatter3d(x=[p2[0]], y=[p2[1]], z=[p2[2]], mode='markers', 
                                   marker=dict(size=8, color='magenta'), name='Senke (berechnet)'))
        
        # Hinzufügen von Text "Spitze" und "Senke"
        fig.add_trace(go.Scatter3d(x=[p1[0]], y=[p1[1]], z=[p1[2]], mode='text', 
                                   text=['Spitze (berechnet)'], textposition="top center", textfont=dict(size=10), showlegend=False))
        fig.add_trace(go.Scatter3d(x=[p2[0]], y=[p2[1]], z=[p2[2]], mode='text', 
                                   text=['Senke (berechnet)'], textposition="top center", textfont=dict(size=10), showlegend=False))
        
    # Einzeichnen der Kollisionen in rot
    if plot_collisions == True and collision_list is not None:
        first_i_for_legend = None
        for i, rect in enumerate(hook_rectangles):      # wenn Index in den Kollisionen enthalten -> rot einzeichnen
            if i in collision_list:
                x, y, z = zip(*rect)
                if first_i_for_legend is None:
                    first_i_for_legend = True      # Legendeeintrag nur für ersten Index zeigen
                
                fig.add_trace(go.Mesh3d(
                    x=x, y=y, z=z,
                    color='red',
                    opacity=1.0,
                    name='Kollision',
                    showlegend=first_i_for_legend
                ))
                first_i_for_legend = False
                
                # Umrandungslinien (optional showlegend=False hier)
                if i == 0 or i == (len(hook_rectangles) - 1):
                    for j in range(4):
                        x0, y0, z0 = rect[j]
                        x1, y1, z1 = rect[(j + 1) % 4]
                        fig.add_trace(go.Scatter3d(
                            x=[x0, x1], y=[y0, y1], z=[z0, z1],
                            mode='lines',
                            line=dict(color='red', width=3),
                            showlegend=False  # keine doppelten Legenden für Linien
                        ))
        
    # Achsen und Layout
    fig.update_layout(
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            aspectmode='cube',      # data
            camera=dict(
                up=dict(x=0, y=-1., z=0),
                eye=dict(x=-0.3, y=-0.3, z=-2)
            )
        ),
        title=f'Haken {hook_num}, Trajektorien-Ansatz {trajectory_process}',
        legend=dict(title="Legende")
    )
    
    # HTML exportieren
    fig.write_html(html_filename)
    print(f"HTML-Datei '{html_filename}' erfolgreich erstellt!")
    # fig.show()

In [199]:
visualize_rectangles_html(
    rectangles = rectangles_1,
    secondary_rectangles = rectangles_optim,
    secondary_legend_title = 'optimale Trajektorie',
    hook_rectangles = rectangles_optim_hooks,
    connect_points = (trajectory_1[1][0], trajectory_1[-1][0]),
    hook_num = hook_num,
    trajectory_process = 1,
    html_filename = '/home/mo/Thesis/Evaluation/Trajektorientests/traj1.html',
    collision_list = collisions_1, plot_collisions = True,
    )

visualize_rectangles_html(
    rectangles = rectangles_2,
    secondary_rectangles = rectangles_optim,
    secondary_legend_title = 'optimale Trajektorie',
    hook_rectangles = rectangles_optim_hooks,
    connect_points = (trajectory_1[1][0], trajectory_1[-1][0]),
    hook_num = hook_num,
    trajectory_process = 2,
    html_filename = '/home/mo/Thesis/Evaluation/Trajektorientests/traj2.html',
    collision_list = collisions_2, plot_collisions = True,
    )

visualize_rectangles_html(
    rectangles = rectangles_3,
    secondary_rectangles = rectangles_optim,
    secondary_legend_title = 'optimale Trajektorie',
    hook_rectangles = rectangles_optim_hooks,
    connect_points = (trajectory_1[1][0], trajectory_1[-1][0]),
    hook_num = hook_num,
    trajectory_process = 3,
    html_filename = '/home/mo/Thesis/Evaluation/Trajektorientests/traj3.html',
    collision_list = collisions_3, plot_collisions = True,
    )

visualize_rectangles_html(
    rectangles = rectangles_4,
    secondary_rectangles = rectangles_optim,
    secondary_legend_title = 'optimale Trajektorie',
    hook_rectangles = rectangles_optim_hooks,
    connect_points = (trajectory_1[1][0], trajectory_1[-1][0]),
    hook_num = hook_num,
    trajectory_process = 4,
    html_filename = '/home/mo/Thesis/Evaluation/Trajektorientests/traj4.html',
    collision_list = collisions_4, plot_collisions = True,
    )

/tmp/ipykernel_13801/2795630801.py:19: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.



HTML-Datei '/home/mo/Thesis/Evaluation/Trajektorientests/traj1.html' erfolgreich erstellt!
HTML-Datei '/home/mo/Thesis/Evaluation/Trajektorientests/traj2.html' erfolgreich erstellt!
HTML-Datei '/home/mo/Thesis/Evaluation/Trajektorientests/traj3.html' erfolgreich erstellt!
HTML-Datei '/home/mo/Thesis/Evaluation/Trajektorientests/traj4.html' erfolgreich erstellt!
